<a href="https://colab.research.google.com/github/DavidIzadaR/Google-Earth-Engine/blob/main/Google_Earth_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import the earth-engine module
import ee

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=t8Tj4NyYPWZ26Y5cIG0XE_EXsu9Q9KEJsxHtvzAjUQs&tc=8Ejduq14pr7DIw2WRvGJuXv1iQm4KXD0V1rFEjV5XJA&cc=Whew8t-BNdJjE_ZqhaaMBY3PReVnRMi837GLcWs-1cM

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjaXBwJtudAFqswiXgnqkmLu-Mk5w9gMhP_VYMA4qEOR7VsbAiBcvE

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
# Load the Nighttime Image Collection
 
collection = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS')

In [5]:
# Extract the first image
first_image = collection.first()
 
# Display band information about the image
first_image.bandNames().getInfo()

['avg_vis', 'stable_lights', 'cf_cvg', 'avg_lights_x_pct']

In [6]:
collection = collection.select('stable_lights')

In [7]:
first_image = collection.first()
 
# Get number of years after 1991 from the first image
year = ee.Date(first_image.get('system:time_start')).get('year').subtract(1991)
 
# Turn it into a band
img1 = ee.Image(year).byte().addBands(first_image)
 
# A new band appears in the new image
img1.bandNames().getInfo()

['constant', 'stable_lights']

In [8]:
def createTimeBand(img):
    year = ee.Date(img.get('system:time_start')).get('year').subtract(1991)
    return ee.Image(year).byte().addBands(img)
 
# Map it to the collection to create a new collection
collection = collection.map(createTimeBand)

In [9]:
#pip install folium --upgrade

# Import the folium library
import folium

In [10]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)
 
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [11]:
# Create the night map
night_map = folium.Map(location=[37.5010, -102.1899], zoom_start=4.5)

In [12]:
#  visualize the y-intercept in green, and positive/negative slopes as red/blue.
 
params = {
    'bands' :  ['scale', 'offset', 'scale'],
    'min' : 0,
    'max' : [0.18, 20, -0.18],
}

In [13]:
night_light = collection.reduce(ee.Reducer.linearFit())

In [16]:
# Add the image layer to the night_map
night_map.add_ee_layer(night_light, params, 'test')
 
# Display the map
display(map)

map